In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import MisrToolkit as mtk # needs python3.6
# from mpl_toolkits.basemap import Basemap # needs python 3.8?
import sys # to check python version
import os
import glob
import datetime as dt

In [2]:
# check python version
sys.version

'3.6.9 (default, Dec  8 2021, 21:08:43) \n[GCC 8.4.0]'

In [3]:
# load training dataset
# ds_fp = "/Users/ehsanmos/Documents/RnD/MISR_lab/ML_research/training_dataset/merged_april_2016_9cam3bands.csv"
# home_dir = 

# ds_fp = "/home/ehsan/misr_lab/training_dataset/april_2016/april_2016_parts/merged_april_2016_9cam_4bands_final_dataset_2.csv"
ds_fp = "/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/training_data/MisrAtmCalibratedModel_april_2016_testRun.csv"

in_ds = pd.read_csv(ds_fp)

In [4]:
in_ds.head(5)

,#path,orbit,img_block,line,sample,lat,lon,an,ca,cf,rms,weight,npts,cloud,var
0,79,86932,32,411,771,71.753167,203.918390,0.654443,0.755881,0.791431,12.678750,0.5,8.0,-1,1.827518
1,79,86932,32,411,772,71.750189,203.921959,0.666452,0.778317,0.776579,12.010000,0.5,0.5,-1,0.000000
2,79,86932,32,412,772,71.749996,203.922187,0.676406,0.780845,0.787165,8.312353,0.5,8.5,-1,1.919447
3,79,86932,32,413,773,71.746816,203.925895,0.697736,0.813235,0.787481,9.718824,0.5,8.5,-1,3.094357
4,79,86932,32,414,773,71.743552,203.929087,0.713062,0.790641,0.801069,16.385000,0.5,1.0,-1,0.305000


In [5]:
# remove lines with zero
# Note: you must put each condition within parenthesis

# we use 3 filters to make sure we find every zero row and copy the filterd DF to new one
in_ds = in_ds.loc[(in_ds['#path']!=0) &
                 (in_ds[' orbit'] !=0) &
                 (in_ds[' img_block'] !=0)]
in_ds.head()

,#path,orbit,img_block,line,sample,lat,lon,an,ca,cf,rms,weight,npts,cloud,var
0,79,86932,32,411,771,71.753167,203.918390,0.654443,0.755881,0.791431,12.678750,0.5,8.0,-1,1.827518
1,79,86932,32,411,772,71.750189,203.921959,0.666452,0.778317,0.776579,12.010000,0.5,0.5,-1,0.000000
2,79,86932,32,412,772,71.749996,203.922187,0.676406,0.780845,0.787165,8.312353,0.5,8.5,-1,1.919447
3,79,86932,32,413,773,71.746816,203.925895,0.697736,0.813235,0.787481,9.718824,0.5,8.5,-1,3.094357
4,79,86932,32,414,773,71.743552,203.929087,0.713062,0.790641,0.801069,16.385000,0.5,1.0,-1,0.305000


In [6]:
in_ds.shape

(120969, 15)

In [7]:
# ds_3cam = in_ds[["path", "orbit", "block", "line", "sample", "lat", "lon", "Ca_r", "Cf_r", "An_r", "mean_ATM_roughness"]] # add path & block here

ds_3cam = in_ds[["#path", " orbit", " img_block", " line", " sample", " lat", " lon", " an", " ca", " cf", " rms"]] # add path & block here


In [8]:
ds_3cam.rename(columns={"mean_ATM_roughness":"rms"}, inplace=True)
ds_3cam.columns

/home/ehsan/.local/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Index(['#path', ' orbit', ' img_block', ' line', ' sample', ' lat', ' lon',
       ' an', ' ca', ' cf', ' rms'],
      dtype='object')

In [9]:
# # determine range to print based on min, max lat and lon of the data
# # is it necessary?
# xlon = ds_3cam[' lon']
# ylat = ds_3cam[' lat']


# margin = 2 # buffer to add to the range
# lat_min = min(ylat) - margin
# lat_max = max(ylat) + margin
# lon_min = min(xlon) - margin
# lon_max = max(xlon) + margin

In [10]:
# print(lat_min)
# print(lat_max)
# print(lon_min)
# print(lon_max)

In [11]:
# for plotting needed # from itertools import chain

# def draw_map(m, scale=0.2):
#     # draw a shaded-relief image
#     m.shadedrelief(scale=scale)
    
#     # lats and longs are returned as a dictionary
#     lats = m.drawparallels(np.linspace(-90, 90, num=7))
#     lons = m.drawmeridians(np.linspace(-180, 180, num=18))

#     # keys contain the plt.Line2D instances
#     lat_lines = chain(*(tup[1][0] for tup in lats.items()))
#     lon_lines = chain(*(tup[1][0] for tup in lons.items()))
#     all_lines = chain(lat_lines, lon_lines)
    
#     # cycle through these lines and set the desired style
#     for line in all_lines:
#         line.set(linestyle='-', alpha=0.3, color='w')

In [12]:
# ref: https://stackoverflow.com/questions/44488167/plotting-lat-long-points-using-basemap
# basemap = Basemap(llcrnrlon=lon_min,
#                     llcrnrlat=lat_min,
#                     urcrnrlon=lon_max,
#                     urcrnrlat=lat_max,
#                     lat_0=(lat_max - lat_min)/2,
#                     lon_0=(lon_max-lon_min)/2,
#                     projection='ortho',
#                     resolution = None,
#                     area_thresh=10000.,)

In [13]:
# basemap.shadedrelief()
# basemap.drawcoastlines(color='gray')
# basemap.drawcountries(color='gray')
# basemap.drawstates(color='gray')



# basemap.drawcoastlines()
# basemap.drawcountries()
# basemap.drawstates()
# basemap.drawmapboundary(fill_color='#46bcec')
# basemap.fillcontinents(color = 'white',lake_color='#46bcec')

In [14]:
# fig = plt.figure(figsize=(12, 12))

# basemap = Basemap(projection='ortho', resolution=None, lat_0=90, lon_0=180)

# # locations on map: convert lat and lon to map projection coordinates
# xlons, ylats = basemap(xlon, ylat)

# plt.plot(xlons, ylats, 'ok', markersize=1)

# # draw a circle on map on 60N  
# basemap.drawparallels([60], color='red') #, labels='test'????)
# plt.title('Study Region, 60N', fontsize=16)

# draw_map(basemap);

# how zoom to the region?

In [15]:
# needed for plotting
# def make_the_map(XlonSet, YlatSet):
    
#     fig = plt.figure(figsize=(12, 12))

#     basemap = Basemap(projection='ortho', resolution=None, lat_0=90, lon_0=180)

#     # locations on map: convert lat and lon to map projection coordinates
#     xlons, ylats = basemap(XlonSet, YlatSet)

#     plt.plot(xlons, ylats, 'ok', markersize=1)

#     # draw a circle on map on 60N  
#     basemap.drawparallels([60], color='red') #, labels='test'????)
#     plt.title('Study Region, 60N', fontsize=16)

#     draw_map(basemap);
#     return 0;

In [16]:
# run with oython >= 3.8

# make_the_map(xlon, ylat)

In [17]:
ds_3cam[' orbit']

0         86932
1         86932
2         86932
3         86932
4         86932
          ...  
120964    86916
120965    86916
120966    86916
120967    86916
120968    86916
Name:  orbit, Length: 120969, dtype: int64

In [18]:
# compute time from orbit

orbit_list = ds_3cam[' orbit'].to_list()
print(type(orbit_list))

# for x in orbit_list:
#     if x<1:
#         orbit_list.remove(x)

orbit_list_noZero = [i for i in orbit_list if i>0]

# print(len(orbit_list_noZero))
# print(min(orbit_list_noZero))
# print(max(orbit_list_noZero))

<class 'list'>


In [19]:
# use mtk and change orbit to date-time

orbit_start_stop_list = []
for orbiti in orbit_list_noZero:
    orbit_start_stop = mtk.orbit_to_time_range(orbiti)
    orbit_start_stop_list.append(orbit_start_stop)

In [20]:
# change list to np.array and extract start and stop columns 

orbit_start = np.array(orbit_start_stop_list)[:,0]
orbit_stop = np.array(orbit_start_stop_list)[:,1]

In [21]:
# orbit_start

In [22]:
# add a date-time column to DF

ds_3cam['orbit_start'] = orbit_start
ds_3cam['orbit_stop'] = orbit_stop

/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [23]:
# change string to datetime format
# notes: String to datetime object = strptime & datetime object to other formats = strftime

# orbit_start[0]
# dt_format = "%Y-%m-%dT%H:%M:%SZ"
# dt.datetime.strptime(orbit_start, dt_format)

In [24]:
# change string to datetime format and apply it on specific DF column

ds_3cam['orbit_start'] = pd.to_datetime(ds_3cam['orbit_start'])
ds_3cam['orbit_stop'] = pd.to_datetime(ds_3cam['orbit_stop'])
ds_3cam.head()

/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ehsan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,#path,orbit,img_block,line,sample,lat,lon,an,ca,cf,rms,orbit_start,orbit_stop
0,79,86932,32,411,771,71.753167,203.918390,0.654443,0.755881,0.791431,12.678750,2016-04-21 22:01:26+00:00,2016-04-21 23:40:20+00:00
1,79,86932,32,411,772,71.750189,203.921959,0.666452,0.778317,0.776579,12.010000,2016-04-21 22:01:26+00:00,2016-04-21 23:40:20+00:00
2,79,86932,32,412,772,71.749996,203.922187,0.676406,0.780845,0.787165,8.312353,2016-04-21 22:01:26+00:00,2016-04-21 23:40:20+00:00
3,79,86932,32,413,773,71.746816,203.925895,0.697736,0.813235,0.787481,9.718824,2016-04-21 22:01:26+00:00,2016-04-21 23:40:20+00:00
4,79,86932,32,414,773,71.743552,203.929087,0.713062,0.790641,0.801069,16.385000,2016-04-21 22:01:26+00:00,2016-04-21 23:40:20+00:00


In [25]:
# extract transect by date
# filter for 19-april and then plot the transect for both my method and old method and then compare...

transect_19apr = ds_3cam.loc[(ds_3cam['orbit_start'] >= '2016-04-19 00:00:00') & (ds_3cam['orbit_stop'] <= '2016-04-19 06:00:0')]
transect_19apr.head()



,#path,orbit,img_block,line,sample,lat,lon,an,ca,cf,rms,orbit_start,orbit_stop
2257,106,86890,13,476,724,79.833672,273.193115,0.618979,0.773878,1.058473,8.145106,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
2258,106,86890,13,477,724,79.834485,273.193192,0.614676,0.770804,1.059088,8.126279,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
2259,106,86890,13,477,723,79.835840,273.193339,0.619594,0.779410,1.058473,8.103906,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
2260,106,86890,13,478,723,79.838281,273.193639,0.617750,0.772034,1.059702,7.265312,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
2261,106,86890,13,478,722,79.839366,273.193783,0.622053,0.781254,1.049867,9.324225,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00


In [26]:
transect_19apr[' lon'].min()

203.03163899999998

In [27]:
# extract transect by guessing longitudes
transect_19apr = transect_19apr.loc[(transect_19apr[' lon'] > 272.0) & (transect_19apr[' lon'] < 273.0)]
transect_19apr


,#path,orbit,img_block,line,sample,lat,lon,an,ca,cf,rms,orbit_start,orbit_stop
18327,106,86890,14,64,574,80.154877,272.998852,0.639937,0.800221,1.072164,7.507500,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
18328,106,86890,14,65,574,80.157872,272.996254,0.638736,0.797820,1.066761,7.442308,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
18329,106,86890,14,65,573,80.158689,272.995535,0.636935,0.797219,1.069763,7.525161,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
18330,106,86890,14,66,573,80.160867,272.993595,0.638136,0.798420,1.069763,7.427083,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
18331,106,86890,14,66,572,80.162501,272.992122,0.638736,0.796019,1.074565,6.714091,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107216,106,86890,14,35,610,80.042727,272.941903,0.634135,0.779306,1.095159,6.995000,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
107217,106,86890,14,34,610,80.041755,272.954658,0.629276,0.777484,1.098196,7.927778,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
107218,106,86890,14,33,609,80.039716,272.979919,0.637780,0.782343,1.095766,7.260000,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
107219,106,86890,14,32,609,80.039584,272.981487,0.636565,0.779306,1.099411,7.297778,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00


In [28]:
# create a column as index and set that column as index to plot transect

# lon_subset['index'] = [i for i in range(len(lon_subset['lon']))]
transect_19apr.index = range(1,len(transect_19apr)+1)
transect_19apr.head()

# set that column as index
# lon_subset.set_index['index']

,#path,orbit,img_block,line,sample,lat,lon,an,ca,cf,rms,orbit_start,orbit_stop
1,106,86890,14,64,574,80.154877,272.998852,0.639937,0.800221,1.072164,7.507500,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
2,106,86890,14,65,574,80.157872,272.996254,0.638736,0.797820,1.066761,7.442308,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
3,106,86890,14,65,573,80.158689,272.995535,0.636935,0.797219,1.069763,7.525161,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
4,106,86890,14,66,573,80.160867,272.993595,0.638136,0.798420,1.069763,7.427083,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
5,106,86890,14,66,572,80.162501,272.992122,0.638736,0.796019,1.074565,6.714091,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00


In [29]:
# select a smaller subset of transect, like the first 1000 points (or 10)

# transect_small = transect.iloc[0:10000]
transect_small = transect_19apr
transect_small.head()

,#path,orbit,img_block,line,sample,lat,lon,an,ca,cf,rms,orbit_start,orbit_stop
1,106,86890,14,64,574,80.154877,272.998852,0.639937,0.800221,1.072164,7.507500,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
2,106,86890,14,65,574,80.157872,272.996254,0.638736,0.797820,1.066761,7.442308,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
3,106,86890,14,65,573,80.158689,272.995535,0.636935,0.797219,1.069763,7.525161,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
4,106,86890,14,66,573,80.160867,272.993595,0.638136,0.798420,1.069763,7.427083,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00
5,106,86890,14,66,572,80.162501,272.992122,0.638736,0.796019,1.074565,6.714091,2016-04-19 00:48:19+00:00,2016-04-19 02:27:12+00:00


In [30]:
# needed for plotting
# xlon270 = transect_small[' lon']
# ylat270 = transect_small[' lat']

In [31]:
# run with python >= 3.8

# make_the_map(xlon270, ylat270)

In [32]:
# extract transect data write it out

# output_filename = "insample_atm_withPathOrbit_3cams_forTransect_mymethod_19apr_hr0-6_lon272-273.csv"
output_filename = "insample_atm_withPathOrbit_3cams_forTransect_oldCodeMethod_19apr_hr0-6_lon272-273.csv"

# output_path = "/Users/ehsanmos/Documents/RnD/MISR_lab/ML_research/training_dataset"
output_path = "/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/training_data"

transect_small.to_csv(os.path.join(output_path, output_filename))